In [ ]:
import requests as req
import pandas as pd
import gzip
import re
import os

In [ ]:
os.getcwd()

'/content'

In [ ]:
#file downloading
url = "http://nldslab.soe.ucsc.edu/iac/v2/convinceme_2016_05_18.sql.gz"
r = req.get(url)
name = url.split("/")[-1] #createdebate_released_no_parse_2016_05_18.sql.gz
with open(name,'wb') as f:
  f.write(r.content)
f.close()
#Extracting .gz file into sql
sql = open(name[:-3],'wb') #createdebate_released_no_parse_2016_05_18.sql
with gzip.open('/content/'+name, 'rb') as f:
    sql.write(f.read())
f.close()
sql.close()
os.remove("/content/"+name)
name = name[:-3] #createdebate_released_no_parse_2016_05_18.sql

In [ ]:
datafile = open(name,'r')
data = datafile.readlines()
datafile.close()
print(len(data))
com = []
for i in range(len(data)):
  if data[i][0:2] not in ['--','/*','\n','DR','LO','UN']:
    com+=[data[i]]
print(len(com))
print(com)

1404
936


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
DictofDF = {}
#flags
tabcre = None
for i in range(len(com)):
  val = com[i]
  if 'CREATE TABLE' in val:
    tname = re.search(r'`[^\s()]+`',val)
    tname = tname.group()[1:-1]
    DictofDF[tname] = pd.DataFrame()
    print(DictofDF.keys())
    tabcre = tname
    continue
  if tabcre != None:
    if 'PRIMARY' in val:
      tabcre = None
    else:
      tcol = re.search(r'`[^\s()]+`',val)
      tcol = tcol.group()[1:-1]
      DictofDF[tabcre][tcol] = ""
      print("datframe",DictofDF)
  if 'INSERT INTO' in val:
    tab = re.search(r'`[^\s()]+`',val)
    tab = tab.group(0)[1:-1]
    print(tab)
    Df = DictofDF[tab]
    val = val[val.find('('):].split("),")  #split from left parantheses to right parantheses with ,split
    print(Df.columns)
    #print(val[0])
    for row in range(len(val)):
      lis = val[row][1:].split(',') #split the row with comma and take the values as list
      if len(lis)!=len(Df.columns):
        lis = lis[0:len(Df.columns)-1]+[str(lis[len(Df.columns):])]
        lis[-1]= lis[-1][1:-1]
        #Df.loc[len(Df)]=lis
      #if len(lis)==len(Df.columns):
      Df.loc[len(Df)]=lis
DictofDF.keys()


dict_keys(['author'])
datframe {'author': Empty DataFrame
Columns: [author_id]
Index: []}
datframe {'author': Empty DataFrame
Columns: [author_id, username]
Index: []}
author
Index(['author_id', 'username'], dtype='object')
dict_keys(['author', 'corenlp_coref'])
datframe {'author':      author_id                 username
0            1       'kittycatmeowmeow'
1            2                'pgandal'
2            3            'iknowindigo'
3            4                'vonkain'
4            5             'Adam Green'
...        ...                      ...
5778      5779          'Peter Jussila'
5779      5780        'Ciarán Harkness'
5780      5781         'Felipe Velasco'
5781      5782         'Rich Dellinger'
5782      5783  'Samuel Hermansson');\n

[5783 rows x 2 columns], 'corenlp_coref': Empty DataFrame
Columns: [text_id]
Index: []}
datframe {'author':      author_id                 username
0            1       'kittycatmeowmeow'
1            2                'pgandal'
2       

In [ ]:
for i in DictofDF.keys():
  print(DictofDF[i].head())

In [ ]:
from pandas import ExcelWriter
writer = ExcelWriter('/content/'+name[:-4]+'.xlsx')
for key in DictofDF.keys():
  DictofDF[key].to_excel(writer, sheet_name=key)
writer.save()